<a href="https://colab.research.google.com/github/itslastonenikhil/federated-learning/blob/main/FederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Packages

In [ ]:
import torch
import math
import plotly.express as px
import pandas as pd
import numpy as np
from copy import deepcopy
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import DataLoader, random_split, TensorDataset
# Dataset stores samples and corresponding labels 
# DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

### Creating Nodes

#### Ploting Client Samples

In [ ]:
def plot_grid(data, samples = 25):
    dim = int(samples/5)


    for i in range(len(data)):
        
        if(i == 3):
            break

        title = "Client " + str(i+1)
        fig = plt.figure(figsize=(dim, dim))
        plt.suptitle(title)
        X, y =  next(iter(data[i]))
            
        for j in range(samples):
            fig.add_subplot(dim, 5, j+1)
            image = X[j][0]
            plt.imshow(image, cmap='gray')
            plt.axis('off')


#### Client's Datastuctures

    Client - 3 (list)
    - iid_train[0]
    - iid_train[1]
    - iid_train[2]

    iter(iid_train[0]) gives an iterator to 8 batches(size 25)

    iid_train[
        client--> 0/2
        [
            batch --> 0/8
            [
                25 images tensor --> [], 
                25 labels tensor --> []
            ],[[[], []],... , [[[], []]],
        client--> 1/2
        [[],[],[],[],[],[],[],[]], 
        client--> 2/2
        [[],[],[],[],[],[],[],[]]
    ]


In [ ]:
train_dataset = torchvision.datasets.MNIST(
        root = './data',
        train=True,
        download=True,
        transform = torchvision.transforms.ToTensor()
    )

test_dataset = torchvision.datasets.MNIST(
    root = './data',
    train = False,
    download = True,
    transform = torchvision.transforms.ToTensor()
)

In [ ]:
from flask import Flask, request, jsonify
import threading
import time

app = Flask(__name__)

clients = []
registration_open = True

@app.route("/register", methods=["POST"])
def register_client():
    if registration_open:
        client_ip = request.remote_addr
        data = request.get_json()
        client_port = data["port"]

        client_addr = (client_ip, client_port) 

        if client_addr not in clients:
            clients.append(client_addr)
    else:
        return jsonify({"message": "Registration closed!"}), 403

    return jsonify({"message": "Client registered successfully!", "clients": clients})

@app.route("/get_clients", methods=["GET"])
def get_clients():
    return jsonify({"clients": clients})


# @app.route("/get_data", methods=["GET"])
# def get_data():
#     client_ip = request.remote_addr
#     if client_ip not in client_data_map:
#         return jsonify({"message": "Client is not registered or has no assigned data."}), 404
    
#     payload = client_data_map[client_ip]

#     return jsonify(payload)

def close_registration(registration_timeout):
    global registration_open

    time.sleep(registration_timeout)
    registration_open = False
    print("REGISTRATION CLOSED")

def run_flask():
    app.run(host="0.0.0.0", port=5000)

registration_timeout = 30

threading.Thread(target=close_registration, args=(registration_timeout,), daemon=True).start()
threading.Thread(target=run_flask, daemon=True).start()

In [ ]:
import requests

response = requests.get("http://localhost:5000/get_clients").json()
print(response)

#CLIENT_LIST HOLDS LIST OF CLIENT IP ADDRESSES
client_list = response['clients']

In [ ]:
from torch.utils.data import Subset

## ONLY USING 1% OF DATASET
total_train_size = len(train_dataset) // 100 + 2
total_test_size = len(test_dataset) // 100

# train_dataset only hold subset of full train dataset
subset_indices = list(range(0, total_train_size))
train_dataset = Subset(train_dataset, subset_indices)

# test_dataset only hold subset of full test dataset
subset_indices = list(range(0, total_test_size))
test_dataset = Subset(test_dataset, subset_indices)

classes = 10
input_dim = 784

num_train_samples = 5000
num_test_samples = 1000
shuffle = True

num_clients = len(client_list)
rounds = 30
# batch_size = 128
batch_size = 64
epochs_per_client = 3
learning_rate = 2e-2


In [ ]:
train_dataset = torchvision.datasets.MNIST(
        root = './data',
        train=True,
        download=True,
        transform = torchvision.transforms.ToTensor()
    )

test_dataset = torchvision.datasets.MNIST(
    root = './data',
    train = False,
    download = True,
    transform = torchvision.transforms.ToTensor()
)

#### IID Splitting

In [ ]:
client_data_map = {}
# NETWORK LOGIC
def push_data():
    print(f"Client Map: {len(client_data_map)}")
    for client_addr, payload in client_data_map.items():
        client_ip, client_port = client_addr
        train_payload, test_payload = payload

        print(f"PAYLOAD: {train_payload.keys()}")
        print(f"PAYLOAD: {test_payload.keys()}")
        try:
            for i in range(2):
                url = f"http://{client_ip}:{client_port}/send_data"
                response = requests.post(url, json=payload[i], timeout=5)

                if response.status_code == 200:
                    print(f"Data successfully transferred to client {(client_ip, client_port)}")
                else:
                    print(f"Failed to send data to client {(client_ip, client_port)}")

        except Exception as e:
            print(f"Error sending data to {(client_ip, client_port)}: {e}")

def iid_uniform(dataset, num_clients, total_train_size, batch_size, is_train_data, shuffle=True):
    print(f"Number of clients: {num_clients}")
    samples_per_client = int(total_train_size / num_clients)
    print(f"Total train size: {total_train_size}")
    print(f"Number of Samples: {samples_per_client}")

    # wrap an iterable around the dataset
    loader = DataLoader(dataset, batch_size=samples_per_client, shuffle=shuffle)    # batch per client

    itr = iter(loader)
    data = []


    for i in range(num_clients):
        # takes next batch, unpacks it to (features, label), and makes into mini dataset via TensorDataset. 
        # PRODUCES DATALOADER FOR A CLIENT


        # SEND DATA TO RESPECTIVE CLIENTS!
        batch = next(itr)
        features, labels = batch

        node_dataloader = DataLoader(TensorDataset(*batch), batch_size=batch_size, shuffle=shuffle)    # each client is further broken down to more batches
        
        payload = {"features" : features.tolist(), "labels": labels.tolist()}
        
        client_ip = client_list[i]

        if client_ip not in client_data_map:
            client_data_map[client_ip] = [None, None]

        if is_train_data:
            payload['is_train_data'] = True
            client_data_map[client_ip][0] = payload
        else:
            payload['is_train_data'] = False
            client_data_map[client_ip][1] = payload

        data.append(node_dataloader)

    # returns list of iterable forms of client datasets
    return data            


In [ ]:
def plot_iid_distribution(data_loaders):
    # 1. Bar plot of samples per client
    samples_per_client = [len(loader.dataset) for loader in data_loaders]
    
    plt.figure(figsize=(12, 6))
    
    # Label Distribution
    plt.subplot(1, 2, 2)
    
    labelMap = {j : [0 for i in range(num_clients)] for j in range(10)}     # tracks labels with the number of that label per client
    
    for i in range(len(data_loaders)):
        loader = data_loaders[i]
        for _, y in loader.dataset:
            label = y.item()
            labelMap[label][i] += 1
    
    x = range(num_clients)
    bottom = np.zeros(num_clients)

    colors = plt.cm.get_cmap('tab10')(np.linspace(0, 1, 10))

    for label in range(10):
        plt.bar(x, labelMap[label], bottom=bottom, label=f'Label {label}', 
                color=colors[label])
        bottom += np.array(labelMap[label])
    
    plt.title('Label Distribution Across Clients')
    plt.xlabel('Client ID')
    plt.ylabel('Number of Samples')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

# Usage example:

# distributed_data = iid_uniform(train_dataset, num_clients, num_train_samples, batch_size, True)

# plot_iid_distribution(distributed_data)

#### Non-IID Splitting

In [ ]:
import random

def non_iid_uniform(dataset, numClients, total_train_size, batch_size, is_train_data, shuffle):
    samples_per_client = total_train_size // numClients
    
    print("Creating Non-IID Uniform Split...")
    
    ## HARDCODED FOR MNIST!!!
    num_of_labels = 10

    client_labels = list()  # each element will hold random 30% of labels
    labels_per_client = (int) (num_of_labels * 0.3)

    for i in range(numClients):
        client_labels.append(random.sample(range(0, 9), labels_per_client)) # randomly selects 30% of dataset


    loader = DataLoader(dataset, batch_size=(numClients * samples_per_client), shuffle=shuffle) # 1 batch of the whole dataset
    itr = iter(loader)
    images, labels = next(itr)  # images, labels hold the whole dataset


    data = []

    maxSize = samples_per_client

    image_list, label_list = [], []

    for i in range(numClients):
        is_label_equal = []

        # Check every label in client_labels[i]
        
        for client_label in client_labels[i]:
            is_label_equal.append(client_label == labels)

        index = torch.stack(is_label_equal) # torch.stack combines is_label_equal tensors into 1 tensor
        index = index.sum(0)    # any column that doesn't have 1 will be False, otherwise True
        index = index.bool()    # Converts 0s and 1s back to True/False

        new_images, new_labels = images[index], labels[index]

        ## INDEX ACTS AS A BOOLEAN MASK
        image_list.append(new_images)
        label_list.append(new_labels)

        maxSize = min(maxSize, len(new_images))


    for i in range(len(image_list)):
        new_images, new_labels = image_list[i], label_list[i]

        features, labels = new_images[:maxSize], new_labels[:maxSize]
        payload = {"features": features.tolist(), "labels": labels.tolist()}

        client_ip = client_list[i]

        if client_ip not in client_data_map:
            client_data_map[client_ip] = [None, None]
        if is_train_data:
            payload['is_train_data'] = True
            client_data_map[client_ip][0] = payload
        else:
            payload['is_train_data'] = False
            client_data_map[client_ip][1] = payload

        node_dataloader_ = DataLoader(TensorDataset(features, labels), batch_size=batch_size, shuffle=shuffle)
        data.append(node_dataloader_)

    return data

# non_iid_uniform(train_dataset, num_clients, num_train_samples, batch_size, True)

In [ ]:
def plot_non_iid_distribution(data_loaders):
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 2)
    
    # Create label map for each client
    labelMap = {j: [0 for i in range(len(data_loaders))] for j in range(10)}
    
    # Count labels per client
    for i in range(len(data_loaders)):
        loader = data_loaders[i]
        for _, y in loader.dataset:
            label = y.item()
            labelMap[label][i] += 1
    
    # Create stacked bar chart
    x = range(len(data_loaders))
    bottom = np.zeros(len(data_loaders))
    
    # Use distinct colors for each label
    colors = plt.cm.get_cmap('tab10')(np.linspace(0, 1, 10))
    
    for label in range(10):
        plt.bar(x, labelMap[label], bottom=bottom, 
                label=f'Label {label}', 
                color=colors[label])
        bottom += np.array(labelMap[label])
    
    plt.title('Label Distribution Across Clients (Non-IID)')
    plt.xlabel('Client ID')
    plt.ylabel('Number of Samples')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

# Usage:
# train_non_iid_data = non_iid_uniform(train_dataset, num_clients, num_train_samples, batch_size, True)
# plot_non_iid_distribution(train_non_iid_data)

# test_non_iid_data = non_iid_uniform(test_dataset, num_clients, num_train_samples, batch_size, True)
# plot_non_iid_distribution(test_non_iid_data)

In [ ]:
def non_iid_skewed(dataset, numClients, total_train_size, batch_size, is_train_data, shuffle):
    print("Creating Non-IID Skewed Split...")
    
    ## HARDCODED FOR MNIST!!!
    num_of_labels = 10

    client_labels = list()  # each element will hold random 30% of labels
    labels_per_client = (int) (num_of_labels * 0.3)

    for i in range(numClients):
        client_labels.append(random.sample(range(0, 9), labels_per_client)) # randomly selects 30% of dataset


    loader = DataLoader(dataset, batch_size=(total_train_size), shuffle=shuffle) # 1 batch of the whole dataset
    itr = iter(loader)
    images, labels = next(itr)  # images, labels hold the whole dataset


    data = []
    
    image_list, label_list = [], []

    for i in range(numClients):
        is_label_equal = []

        # Check every label in client_labels[i]
        
        for client_label in client_labels[i]:
            is_label_equal.append(client_label == labels)

        index = torch.stack(is_label_equal) # torch.stack combines is_label_equal tensors into 1 tensor
        index = index.sum(0)    # any column that doesn't have 1 will be False, otherwise True
        index = index.bool()    # Converts 0s and 1s back to True/False

        new_images, new_labels = images[index], labels[index]

        ## INDEX ACTS AS A BOOLEAN MASK
        image_list.append(new_images)
        label_list.append(new_labels)


    proportionsPerClient = [0.5, 0.25, 0.15, 0.10]
    for i in range(len(image_list)):
        new_images, new_labels = image_list[i], label_list[i]

        clientSize = int(total_train_size * proportionsPerClient[i])

        features, labels = new_images[:clientSize], new_labels[:clientSize]

        payload = {"features": features.tolist(), "labels": labels.tolist()}
        client_ip = client_list[i]

        if client_ip not in client_data_map:
            client_data_map[client_ip] = [None, None]
        if is_train_data:
            payload['is_train_data'] = True
            client_data_map[client_ip][0] = payload
        else:
            payload['is_train_data'] = False
            client_data_map[client_ip][1] = payload

        node_dataloader_ = DataLoader(TensorDataset(features, labels), batch_size=batch_size, shuffle=shuffle)
        data.append(node_dataloader_)

    push_data()

    return data

# train_non_iid_skewed = non_iid_skewed(train_dataset, num_clients, num_train_samples, batch_size, True)
# plot_non_iid_distribution(train_non_iid_skewed)

# test_non_iid_skewed = non_iid_skewed(test_dataset, num_clients, num_train_samples, batch_size, True)
# plot_non_iid_distribution(test_non_iid_skewed)



#### Loading MNIST Dataset

##### **About MNIST Dataset**

MNIST Handwritten Digit Classification Dataset
The MNIST dataset is an acronym that stands for the Modified National Institute of Standards and Technology dataset.

It is a dataset of 60,000 small square 28×28 pixel grayscale images of handwritten single digits between 0 and 9. The task is to classify a given image of a handwritten digit into one of 10 classes representing integer values from 0 to 9, inclusively.

It is a widely used and deeply understood dataset and, for the most part, is “solved.”

In [ ]:
def get_MNIST(type='iid', n_samples_train = 200, n_samples_test= 100, n_clients=3, batch_size=25, shuffle=True):

    # Download MNIST train and test dataset

    train_dataset = torchvision.datasets.MNIST(
        root = './data',
        train=True,
        download=True,
        transform = torchvision.transforms.ToTensor()
    )
    print(len(train_dataset))

    test_dataset = torchvision.datasets.MNIST(
        root = './data',
        train = False,
        download = True,
        transform = torchvision.transforms.ToTensor()
    )

    print("Classes: ", train_dataset.classes)
    print("Number of classes: ", len(train_dataset.classes))
    

    if type == "iid":
        print("Train Dataset")
        train = iid_uniform(train_dataset, n_clients, n_samples_train, batch_size, True, shuffle)
        plot_iid_distribution(train)
        print("Test Dataset")
        test = []
        test =  iid_uniform(test_dataset, n_clients, n_samples_test, batch_size, False, shuffle)
        plot_iid_distribution(test)
        push_data()

    elif type == "non_iid":
        print("Train Dataset")
        train = non_iid_uniform(train_dataset, n_clients, n_samples_train, batch_size, True, shuffle)
        plot_non_iid_distribution(train)
        print("Test Dataset")
        test =  non_iid_uniform(test_dataset, n_clients, n_samples_test, batch_size, False, shuffle)
        plot_non_iid_distribution(test)
        push_data()

    elif type == "non_iid_skewed":
        print("Train Dataset")
        train = non_iid_skewed(train_dataset, n_clients, n_samples_train, batch_size, True, shuffle)
        plot_non_iid_distribution(train)
        print("Test Dataset")
        test =  non_iid_skewed(test_dataset, n_clients, n_samples_test, batch_size, False, shuffle)
        plot_non_iid_distribution(test)
        push_data()

    else:
        train = []
        test = []

    # return train, test (AS LIST OF DATALOADERS)
    return train, test

#### Loading Synthetic MNIST Dataset

In [ ]:
!cp  /content/drive/MyDrive/ML/syntheticMNIST.py /content

In [ ]:
!pip install scipy
import syntheticMNIST as syn

In [ ]:
# def synthetic_client_config(type="non_iid", num_clients=3, train_samples=90, test_samples=30):
#     C= {
#      'n_samples_train': train_samples,
#      'font':'DejaVu Sans',
#      'tilt': [0, 45, 90],
#      'std_tilt': 10, #std on the tilt,
#      'seed':0
#      }

#     C['n_samples']= train_samples + test_samples
#     clients = []
#     if(type=="non_iid"):
#         for i in range(num_clients):

#             new_C =deepcopy(C)
#             num_labels = random.randint(2, 5)
#             new_C['numbers'] = random.sample(range(0, 9), num_labels)
#             clients.append(new_C)
#     if(type=="iid"):
#         for i in range(num_clients):

#             new_C =deepcopy(C)
#             new_C['numbers'] = random.sample(range(0, 10), 9)
#             clients.append(new_C)

#     return clients

#### Generating Clients and Datasets

In [ ]:
num_train_samples = 500
num_test_samples = 100
batch_size = 25
shuffle = True

IID Clients

In [ ]:
iid_train, iid_test = get_MNIST(type='iid', n_samples_train = num_train_samples, 
                                n_samples_test= num_test_samples, n_clients=num_clients,
                                batch_size=batch_size, shuffle=shuffle)

iid_client = [iid_train, iid_test]



Non-IID Clients

In [ ]:
niid_train, niid_test = get_MNIST(type="non_iid", n_samples_train = num_train_samples,
                                  n_samples_test= num_test_samples, n_clients=num_clients,
                                  batch_size=batch_size, shuffle=shuffle)
niid_client = [niid_train, niid_test]

Synthetic Non-IID Clients

In [ ]:
# clients_config = synthetic_client_config(type="iid", num_clients=num_clients, 
#                                          train_samples=num_train_samples, 
#                                          test_samples=num_test_samples)

# iid_train_syn, iid_test_syn = syn.get_synth_MNIST(clients_config, batch_size=batch_size, shuffle=shuffle)

# iid_syn_client = [iid_train_syn, iid_test_syn]

In [ ]:
# clients_config = synthetic_client_config(type="non_iid", num_clients=num_clients, 
#                                          train_samples=num_train_samples, 
#                                          test_samples=num_test_samples)

# niid_train_syn, niid_test_syn = syn.get_synth_MNIST(clients_config, batch_size=batch_size, shuffle=shuffle)

# niid_syn_client = [niid_train_syn, niid_test_syn]

#### Plotting Generated Dataset

In [ ]:
# print("IID Synthetic Training Dataset")
# iid_train_syn[0].dataset.plot_samples(0, "Client 1")
# iid_train_syn[1].dataset.plot_samples(0, "Client 2")
# iid_train_syn[2].dataset.plot_samples(0, "Client 3")

In [ ]:
# print("Non-IID Synthetic Training Dataset")
# niid_train_syn[0].dataset.plot_samples(0, "Client 1")
# niid_train_syn[1].dataset.plot_samples(0, "Client 2")
# niid_train_syn[2].dataset.plot_samples(0, "Client 3")

### Models and functions for federated learning algorithms

Features and Classes

In [ ]:
num_features = 28*28
num_classes = 10

Clients

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = x.reshape(-1, 28*28)
        output = self.linear(x)
        return output

In [ ]:
model_0 = LogisticRegression()
model_1 = LogisticRegression()

**Averaging Models Example**

We can get the `state_dicts` of both models, average the parameters and reload the new `state_dict`.

In [ ]:
# Setup
sdA = model_0.state_dict()
sdB = model_1.state_dict()
sdC = {}

# Average all parameters
for key in sdA:
    sdC[key] = (sdB[key] + sdA[key])/2

# Recreate model and load averaged state_dict (or use modelA/B)
model_2 = LogisticRegression()
model_2.load_state_dict(sdC)

#### Functions for Federated Algos

In [ ]:
class History:
    def __init__(self):
        
        self.loss = []       # stores model loss
        self.accuracy = []   # stores model accuracy
        self.model = []      # stores model
        self.variance = []

    def log_server(self, model, client, loss_func):
        # Logging loss to history
        curr_loss = [(float)(get_dataset_loss(model, dataset, loss_func).detach()) for dataset in client[0]]
        # self.client_loss.append(curr_loss)
        self.loss.append(sum(curr_loss)/len(curr_loss))

        # Logging accuracy to history
        curr_acc = [get_dataset_accuracy(model, dataset) for dataset in client[1]]
        # self.client_acc.append(curr_acc)

        client_acc_avg = sum(curr_acc)/len(curr_acc)
        self.accuracy.append(client_acc_avg)

        # Logging model to history
        self.model.append(model.state_dict())


        # Logging variance
        sumVar = 0
        for i in range(len(client[1])):
            sumVar += ((curr_acc[i] - client_acc_avg) ** 2)

        if len(client[1]) == 1:
            self.variance.append(0)
        else:
            self.variance.append(sumVar / (len(client[1]) - 1))

    def log_client(self, model, dataset, loss):
        # Logging loss to history
        self.loss.append(loss)

        # Logging accuracy to history
        curr_acc = get_dataset_accuracy(model, dataset)
        self.accuracy.append(curr_acc)

        # Logging model to history
        self.model.append(model.state_dict())



In [ ]:
def loss_classifier(predictions, labels):

    loss = nn.CrossEntropyLoss(reduction="mean")
    return loss(predictions, labels)


In [ ]:
def get_accuracy(predictions, labels):

     _, predicted = torch.max(predictions, dim=1)
     accuracy = torch.sum(predicted == labels).item()/len(predicted)
     return accuracy


In [ ]:
def get_dataset_loss(model, dataset, loss_func):
    # Compute loss of a model on given dataset

    total_loss = 0
    for batch in dataset:
        features, labels = batch
        predictions = model(features)
        total_loss += loss_func(predictions, labels)

    avg_loss = total_loss/len(dataset)
    return avg_loss

In [ ]:
def get_dataset_accuracy(model, dataset):

    total_accuracy = 0
    for batch in dataset:
        features, labels = batch
        predictions = model(features)
        curr_accuracy = get_accuracy(predictions, labels)
        total_accuracy += curr_accuracy
    
    avg_accuracy = (total_accuracy/len(dataset))*100
    return avg_accuracy


In [ ]:
def train(model, batch, loss_func):
    images, labels = batch
    preds = model(images)
    loss = loss_func(preds, labels)
    return loss


In [ ]:
def diff_squared_sum(model1, model2):
    w1 = model1.linear.weight
    w2 = model2.linear.weight
    w = (w1-w2).pow(2).sum()

    b1 = model1.linear.bias
    b2 = model2.linear.bias
    b = (b1-b2).pow(2).sum()
    
    dss = w + b
   

    return dss


In [ ]:
def get_model_avg(history):

    # Setup
    model_state = history.model
    sd = {k: torch.zeros_like(v) for k, v in model_state[0].items()} # create state dictionary

    # Zero all the values of state dictionary
    for key in sd:
        sd[key] = 0

    # Average all parameters
    n = len(model_state)
    for state in model_state:
        for key in sd:
            sd[key] += state[key]*(1/n)
    
    # Recreate model and load averaged state_dict (or use modelA/B)
    model = LogisticRegression()
    model.load_state_dict(sd)

    return model



#### Federated Algorithms

##### FedAvg and FedProx

In [ ]:
def serialize_model(model):
    state_dict = model.state_dict()
    serialized = {key: value.tolist() for key, value in state_dict.items()}
    return serialized



def Fed(server_model, clients, client_fraction=1, rounds=10, epochs=10, learning_rate=0.01, decay=1, model_type="avg", straggler_percent=0, mu=0, q=0):
    model = deepcopy(server_model)
    loss_func = loss_classifier
    K = len(clients[0])        # number of clients (train dataset)

    server_history = History()
    server_history.log_server(model, clients, loss_func)

    print("Starting...")
    print("Clients:", K, "| Learning Rate:", learning_rate, "| mu:", mu, "| Epochs:", epochs)
    print("Running for", rounds, "rounds |")

    for i in range(rounds):
        # Calculate number of clients with given fraction
        m = max(math.floor(K*client_fraction), 1)

        # Select random set on m clients
        # represents key in client name : client ip
        s = random.sample(range(0, K), m) # for now selecting all clients

        local_history = History()
        server_model = deepcopy(model)

        for j in range(len(s)):
            client_data = client_list[s[j]]
            client_ip, client_port = client_data
            
            url = f"http://{client_ip}:{client_port}/client_update"
            payload = {
                "model": serialize_model(server_model),
                "learning_rate": learning_rate,
                "epochs": epochs,
                "mu": mu,
                "type": model_type
            }
            response = requests.post(url, json=payload)

            client_data = response.json()

            local_history.loss.append(client_data["loss"])
            local_history.accuracy.append(client_data["accuracy"])

            client_state_dict = {k: torch.tensor(v) for k, v in client_data['model'][0].items()}


            local_history.model.append(client_state_dict)

        model = get_model_avg(local_history)
        server_history.log_server(model, clients, loss_func)
        
        # Decrease the learning rate with each rounds
        # learning_rate = learning_rate*decay

        if(i%(rounds/10) == 0):
            print("\b+++")

    hist = server_history
    for i in range(len(hist.loss)):
        if(i%(rounds/10) == 0):
            print("Loss: {:.4f}, Accuracy: {:.4f}".format(hist.loss[i], hist.accuracy[i]))

    print("End")
    
    return model, server_history


##### qFedAvg

In [ ]:
def normal(delta_ws):
   
    w = (delta_ws[0]).pow(2).sum()
    b = (delta_ws[1]).pow(2).sum()

    ss = w + b
    return ss

In [ ]:
def q_aggregate(server_model, deltas, hs):
    num_clients = len(deltas)

    de = np.sum(np.asarray(hs))
    
    # Scale client deltas by multiplying (1/denominator)
    scaled_deltas = []
    for client_delta in deltas:
        print(client_delta[0])
        scaled_deltas.append([(layer * 1.0 / de) for layer in client_delta])

    # Sum scaled client deltas
    sum_delta = deltas[0]
    for i in range(len(scaled_deltas)):
        if(i > 0):
            sum_delta = [(sd+d) for sd, d in zip(sum_delta, scaled_deltas[i])]

 
    # Update server model
    model = deepcopy(server_model)
   

    with torch.no_grad():
        model.linear.weight -= sum_delta[0]
        model.linear.bias -=  sum_delta[1]


    return model
    

In [ ]:
def qFed(server_model, clients, client_fraction=1, rounds=10, epochs=10, learning_rate=0.01, q=0):
    
    model = deepcopy(server_model)
    loss_func = loss_classifier
    K = len(clients[0])        # number of clients

    server_history = History()
    server_history.log_server(model, clients, loss_func)

    print("Starting...")
    print("Clients:", K, "| Learning Rate:", learning_rate, "| q:", q, "| Epochs:", epochs)
    print("Running for", rounds, "rounds |")

    for i in range(rounds):
       
        # Calculate number of clients with given fraction
        m = max(math.floor(K*client_fraction), 1)

        # Select random set on m clients
        s = random.sample(range(0, K), m) # for now selecting all clients
        
        local_history = History()
        s_model = deepcopy(model)


        deltas = []
        hs = []
        for j in range(len(s)):
            client_ip = client_list[s[j]]
            url = f"http://{client_ip}:6001/q_client_update"
            payload = {
                "model": serialize_model(server_model),
                "learning_rate": learning_rate,
                "epochs": epochs,
                "q_val": q
            }
            response = requests.post(url, json=payload)
            client_data = response.json()

            delta = [torch.tensor(layer) for layer in client_data['delta']]
            h = torch.tensor(client_data['h'])
            
            local_history.loss.append(client_data["loss"])
            local_history.accuracy.append(client_data["accuracy"])

            client_state_dict = {k: torch.tensor(v) for k, v in client_data['model'][0].items()}
            local_history.model.append(client_state_dict)

            deltas.append(delta)
            hs.append(h)

        model = q_aggregate(s_model, deltas, hs)
        server_history.log_server(model, clients, loss_func)
    

        if(i%(rounds/10) == 0):
            print("\b+++")

    hist = server_history
    for i in range(len(hist.loss)):
        print("Loss: {:.4f}, Accuracy: {:.4f}".format(hist.loss[i], hist.accuracy[i]))

    print("End")
    
    return model, server_history


#### Plotting Loss and Accuracy

In [ ]:
def plot_loss(loss_list, name_list,  title):
    df = pd.DataFrame(np.column_stack(loss_list), columns=name_list)
    
    fig = px.line(df, 
                x=list(range(0, len(loss_list[0]))),
                y=df.columns[:], 
                labels={
                    "x": "Rounds",
                    "value": "Loss",
                    # "variable": "Clients"
                },
                title=title
            )

    # Show plot 
    fig.show()
    return df

In [ ]:

def plot_acc(acc_list, name_list,  title):
    df = pd.DataFrame(np.column_stack(acc_list), columns=name_list)
    
    fig = px.line(df, 
                x=list(range(0, len(acc_list[0]))),
                y=df.columns[:], 
                labels={
                    "x": "Rounds",
                    "value": "Accuracy",
                    # "variable": "Clients"
                },
                title=title
            )

    # Show plot 
    fig.show()
    return df

In [ ]:
def plot_variance(var_list, name_list, title):
    df = pd.DataFrame(np.column_stack(var_list), columns=name_list)
    
    fig = px.line(df, 
                x=list(range(0, len(var_list[0]))),
                y=df.columns[:], 
                labels={
                    "x": "Rounds",
                    "value": "Variance",
                    # "variable": "Clients"
                },
                title=title
            )

    # Show plot 
    fig.show()
    return df

### Evaluating Models and Algorithms

Pamameters
*   server_model
*   clients 
*   client_fraction =1
*   rounds =10
*   mu = 0
*   epochs = 10
*   learning_rate = 0.01
*   decay = 1
*   type ="avg" ("prox", "q-ffl", "avg")
*   straggler_percent = 0 (0 to 1)
*   q = 0

In [ ]:
model = model_0
rounds = 40
epochs = 25
lr = 0.1

### IID Uniform Testing



In [ ]:
favg_iid_model, favg_iid_hist = Fed(server_model=model, clients=iid_client,
                                    rounds=rounds, epochs=epochs, learning_rate=lr, model_type="avg")



In [ ]:
favg_iid_hist.variance

### Randomized Search Algorithm

In [ ]:
from scipy.stats import uniform

class QFedOptimizer:
    def __init__(self, server_model, clients, client_fraction=1, rounds=10, epochs=10):
        self.server_model = server_model
        self.clients = clients
        self.client_fraction = client_fraction
        self.rounds = rounds
        self.epochs = epochs

    def train_and_evaluate(self, learning_rate, q):
        model, history = qFed(
            server_model=self.server_model,
            clients=self.clients,
            client_fraction=self.client_fraction,
            rounds=self.rounds,
            epochs=self.epochs,
            learning_rate=learning_rate,
            q=q
        )
        # Return final loss and accuracy
        return history.loss[-1], history.accuracy[-1], history.variance[-1]

def optimize_qfed_parameters(server_model, clients):
    optimizer = QFedOptimizer(server_model, clients)
    
    # Define parameter space
    search_space = {
        "learning_rate": uniform(0.001, 0.1),
        "q": uniform(0, 10)
    }
    
    # Track best parameters
    best_loss = float('inf')
    best_params = None
    best_accuracy = 0
    best_variance = float('inf')
    
    # Number of trials
    n_trials = 40
    
    for _ in range(n_trials):
        # Sample parameters
        lr = search_space["learning_rate"].rvs()
        q = search_space["q"].rvs()
        
        # Train and evaluate
        loss, accuracy, variance = optimizer.train_and_evaluate(lr, q)
        
        # Update best parameters BASED ON LOSS
        if loss < best_loss:
            best_variance = variance
            best_loss = loss
            best_accuracy = accuracy
            best_params = {'learning_rate': lr, 'q': q}
            
        print(f"Trial params: lr={lr:.6f}, q={q:.2f}")
        print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    
    return best_params

best_params = optimize_qfed_parameters(server_model=model, clients=iid_client)

best_lr, best_q = best_params.values()
print(best_lr, best_q)

In [ ]:
qffl_iid_model_q, qffl_iid_hist_q = qFed(server_model=model, clients=iid_client,
                                    rounds=rounds, epochs=20, learning_rate=0.1, q=4)

losses = [favg_iid_hist.loss, qffl_iid_hist_q.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_iid_hist.accuracy, qffl_iid_hist_q.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_iid_hist.variance, qffl_iid_hist_q.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)


In [ ]:
best_params = optimize_qfed_parameters(server_model=model, clients=niid_client)

best_lr, best_q = best_params.values()
print(best_lr, best_q)

qffl_niid_model_q0, qffl_niid_hist_q0 = qFed(server_model=model, clients=niid_client,
                                    rounds=rounds, epochs=20, learning_rate=best_lr ,q=best_q)

favg_niid_model, favg_niid_hist = Fed(server_model=model, clients=niid_client,
                                    rounds=rounds, epochs=epochs, learning_rate=lr, model_type="avg")

losses = [favg_niid_hist.loss, qffl_niid_hist_q0.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_niid_hist.accuracy, qffl_niid_hist_q0.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_niid_hist.variance, qffl_niid_hist_q0.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

plot_non_iid_distribution(niid_client[0])

In [ ]:
niid_skewed_clients = get_MNIST(type='non_iid_skewed', n_samples_train = num_train_samples, 
                                n_samples_test= num_test_samples, n_clients=num_clients,
                                batch_size=batch_size, shuffle=shuffle)

best_params = optimize_qfed_parameters(server_model=model, clients=niid_skewed_clients)

best_lr, best_q = best_params.values()
print(best_lr, best_q)

qffl_niid_model_q, qffl_niid__skewed_hist_q0 = qFed(server_model=model, clients=niid_skewed_clients,
                                    rounds=rounds, epochs=20, learning_rate=best_lr ,q=best_q)

favg_niid_skewed_model, favg_niid_skewed_hist = Fed(server_model=model, clients=niid_skewed_clients,
                                    rounds=rounds, epochs=epochs, learning_rate=lr, type="avg")

losses = [favg_niid_skewed_hist.loss, qffl_niid__skewed_hist_q0.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_niid_skewed_hist.accuracy, qffl_niid__skewed_hist_q0.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_niid_skewed_hist.variance, qffl_niid__skewed_hist_q0.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

In [ ]:
qffl_iid_hist_q.variance

### q = 4

In [ ]:
qffl_iid_model_q, qffl_iid_hist_q5 = qFed(server_model=model, clients=iid_client,
                                    rounds=rounds, epochs=20, learning_rate=0.1 ,q=4)

losses = [favg_iid_hist.loss, qffl_iid_hist_q5.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_iid_hist.accuracy, qffl_iid_hist_q5.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_iid_hist.variance, qffl_iid_hist_q5.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

In [ ]:
qffl_iid_hist_q.variance

## Non-IID Uniform Testing

In [ ]:
favg_niid_model, favg_niid_hist = Fed(server_model=model, clients=niid_client,
                                    rounds=rounds, epochs=epochs, learning_rate=lr, type="avg")


In [ ]:
favg_niid_hist.variance

### q = 1.5

In [ ]:
qffl_niid_model_q1, qffl_niid_hist_q1 = qFed(server_model=model, clients=niid_client,
                                    rounds=rounds, epochs=20, learning_rate=0.1 ,q=1.5)

losses = [favg_niid_hist.loss, qffl_niid_hist_q1.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_niid_hist.accuracy, qffl_niid_hist_q1.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_niid_hist.variance, qffl_niid_hist_q1.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

In [ ]:
qffl_niid_hist_q1.variance

## Non-IID Skewed Testing

In [ ]:
niid_skewed_clients = get_MNIST(type='non_iid_skewed', n_samples_train = num_train_samples, 
                                n_samples_test= num_test_samples, n_clients=num_clients,
                                batch_size=batch_size, shuffle=shuffle)

In [ ]:
favg_niid_skewed_model, favg_niid_skewed_hist = Fed(server_model=model, clients=niid_skewed_clients,
                                    rounds=rounds, epochs=epochs, learning_rate=lr, type="avg")

In [ ]:
favg_niid_skewed_hist.variance

### q = 0

In [ ]:
qffl_niid_model_q, qffl_niid__skewed_hist_q0 = qFed(server_model=model, clients=niid_skewed_clients,
                                    rounds=rounds, epochs=20, learning_rate=0.1 ,q=0)

losses = [favg_niid_skewed_hist.loss, qffl_niid__skewed_hist_q0.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_niid_skewed_hist.accuracy, qffl_niid__skewed_hist_q0.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_niid_skewed_hist.variance, qffl_niid__skewed_hist_q0.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

In [ ]:
qffl_niid__skewed_hist_q0.variance

### q = 1

In [ ]:
qffl_niid_model_q, qffl_niid__skewed_hist_q1 = qFed(server_model=model, clients=niid_skewed_clients,
                                    rounds=rounds, epochs=20, learning_rate=0.1 ,q=1.5)

losses = [favg_niid_skewed_hist.loss, qffl_niid__skewed_hist_q1.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_niid_skewed_hist.accuracy, qffl_niid__skewed_hist_q1.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_niid_skewed_hist.variance, qffl_niid__skewed_hist_q1.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

In [ ]:
qffl_niid__skewed_hist_q1.variance

### q = 5

In [ ]:
qffl_niid_skewed_model_q, qffl_niid_skewed_hist_q5 = qFed(server_model=model, clients=niid_skewed_clients,
                                    rounds=rounds, epochs=20, learning_rate=0.1 ,q=5)

losses = [favg_niid_skewed_hist.loss, qffl_niid_skewed_hist_q5.loss]
line_names = ["FedAvg", "qFedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df1 = plot_loss(losses, line_names, title)

acc = [favg_niid_skewed_hist.accuracy, qffl_niid_skewed_hist_q5.accuracy]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_acc(acc, line_names, title)

variance = [favg_niid_skewed_hist.variance, qffl_niid_skewed_hist_q5.variance]
line_names = ["FedAvg", "q-FedAvg(q>0)"]
title = "Comparing different Fed Algorithms on IID Dataset"
df2 = plot_variance(variance, line_names, title)

In [ ]:
df1

In [ ]:
df2